In [1]:
import os

os.environ['POLARS_MAX_THREADS'] = '16'

import polars as pl
import polars_st as st

In [2]:
lf = pl.scan_parquet('data/adresser.pq').select(
    pl.col("vejnavn", "husnr", "postnr", "kommunekode", "landsdelsnuts3"), pl.col("etrs89koordinat_øst").alias("etrs89_east"), pl.col("etrs89koordinat_nord").alias("etrs89_north")
).filter(pl.struct(pl.col("etrs89_east", "etrs89_north")).is_first_distinct()).with_columns(
    pl.struct(pl.col("etrs89_east", "etrs89_north")).hash().alias("address_id").rank("dense").shrink_dtype()
)

lf.head().collect()

vejnavn,husnr,postnr,kommunekode,landsdelsnuts3,etrs89_east,etrs89_north,address_id
str,str,i16,i16,str,f64,f64,u32
"""Bakkenellikevej""","""2""",4583,306,"""DK022""",653487.94,6.2037e6,2548631
"""Valmuestien""","""6""",3460,230,"""DK013""",714572.71,6.1951e6,624411
"""Nyelandsvej""","""78""",2000,147,"""DK011""",721167.76,6.1767e6,1863571
"""Havrevej""","""10""",2970,223,"""DK013""",717609.06,6.1987e6,1831869
"""Tvedgade""","""13H""",6760,561,"""DK032""",485270.27,6.1316e6,300457


In [11]:
gdf = st.GeoDataFrame(lf.collect(engine = 'streaming').with_columns(
    coords = pl.concat_list(pl.col("etrs89_east", "etrs89_north"))
).with_columns(
    geometry = st.point("coords")
).with_columns(
    voronoi = st.voronoi_polygons("geometry")
)).to_geopandas()

gdf

: 